In [ ]:
'''
    takes the hand annotatioons as a csv
    takes target as the string to find 
    finds the hand annot that best matches the target
'''
def find_triplet_in_hand_ann(hand,target):
    max_=-1.0
    for row in hand.iterrows():
        hand_rel = f" {row[1]['subj']} {row[1]['rel']} {row[1]['obj']}"
        sub_h = row[1]['subj']
        cos=vec([hand_rel, target])
        

        if cos > max_:
            max_ = cos
            best=hand_rel
        
        cos=vec([best, hand_rel])
        if sub_h.lower() not in best.lower() or cos < 0.7:
            print(f"Closest match for {hand_rel} is {best} with {cos}")
            continue
        else:
            return best
            #print(f"Closest match for {hand_rel} is {best} with {vec([best, hand_rel])}")
            #print(rouge_([hand_rel], [best]))
            

In [ ]:
import fitz,os
def read_pdf(pdf_file):
    start=False
    sentences=[]
    start_idx=0
    with fitz.open(pdf_file) as pdf_file:
        for page_index, page in enumerate(pdf_file):
            text = page.get_text("text").lower()
            text=text.split(". ")
            sentences.extend(text)
                
    return sentences

'''
    Takes the root dir of the pdfs ("Docs/"), stores all sentences as a list of sentences
'''
def read_files(root_dir):
    
    lines=[]
    for files in os.listdir(root_dir):
        if files[-4:] != '.pdf':
            continue
        sentences = read_pdf(f"{root_dir}/{files}")
        lines.extend(sentences)


    return lines

'''
    computes cos similarity between 2 sentences
'''
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
def vec(sentences):
    # Encode sentences
    embeddings = model.encode([sentences[0], sentences[1]])
    
    # Compute cosine similarity
    similarity = util.cos_sim(embeddings[0], embeddings[1])
    return similarity.item() # Value close to 1 indicates high similarity
sentences = read_files("Docs")